In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("asaniczka/amazon-canada-products-2023-2-1m-products")

print("Path to dataset files:", path)

/Users/yusufsaifutdinov/Desktop/pg project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/yusufsaifutdinov/Desktop/pg project/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/yusufsaifutdinov/.cache/kagglehub/datasets/asaniczka/amazon-canada-products-2023-2-1m-products/versions/15


In [2]:
import re
import nltk
import numpy as np
import pandas as pd
import matplotlib as plt
from tqdm import tqdm_notebook
from ast import literal_eval
from pymystem3 import Mystem
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')

df = pd.read_csv("amz_total_data_limited.csv")

df.head(n = 50)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yusufsaifutdinov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,categoryName,isBestSeller,boughtInLastMonth
0,B07CV4L6HX,"Green Leaf WW3D Wonder Extension Cord Winder, ...",https://m.media-amazon.com/images/I/81cRe0AVC4...,https://www.amazon.ca/dp/B07CV4L6HX,4.4,2876,47.69,0.00,Industrial Scientific,False,0
1,B09N1HGY74,8pcs Toilet Seat Bumpers Universal Toilet Repl...,https://m.media-amazon.com/images/I/512CpB-EP5...,https://www.amazon.ca/dp/B09N1HGY74,3.8,55,10.99,0.00,Industrial Scientific,False,100
2,B087P7538J,YaeCCC 19 Pcs Hole Saw Kit 3/4''(19mm)- 6''(15...,https://m.media-amazon.com/images/I/71P+qLFDFc...,https://www.amazon.ca/dp/B087P7538J,4.0,126,25.99,27.99,Industrial Scientific,False,50
3,B0822FF7YQ,LLPT Butyl Putty Tape White 1 Inch x 33Ft Tigh...,https://m.media-amazon.com/images/I/61wfWLZRn-...,https://www.amazon.ca/dp/B0822FF7YQ,4.5,1936,21.99,30.99,Industrial Scientific,False,100
4,B08NTR8NN4,"Lightbeam 16"" Long Stem Deep Fry Thermometer w...",https://m.media-amazon.com/images/I/61qJG-y1He...,https://www.amazon.ca/dp/B08NTR8NN4,4.2,46,18.99,0.00,Industrial Scientific,False,100
5,B0BYT4SGZV,"etguuds White USB C to USB C Cable [10ft, 2-Pa...",https://m.media-amazon.com/images/I/61NhWv2qfZ...,https://www.amazon.ca/dp/B0BYT4SGZV,4.5,2505,15.99,0.00,Industrial Scientific,False,0
6,B07RQ5389H,REAQER CPAP Hose Holder Hanger for Preventing ...,https://m.media-amazon.com/images/I/71NttUXVLT...,https://www.amazon.ca/dp/B07RQ5389H,4.3,216,27.99,0.00,Industrial Scientific,False,50
7,B0B6VLNR45,"SAVITA 2pcs Magnetic Bottle Openers, Can Tappe...",https://m.media-amazon.com/images/I/51ZK+B6vjS...,https://www.amazon.ca/dp/B0B6VLNR45,4.0,53,9.99,0.00,Industrial Scientific,False,50
8,B08LBYDZ1Q,Simba Lighting LED A19 Light Bulbs (4 Pack) 9W...,https://m.media-amazon.com/images/I/61Ec3IzoRH...,https://www.amazon.ca/dp/B08LBYDZ1Q,4.5,164,15.95,0.00,Industrial Scientific,False,0
9,B07KMT96S2,"Knife Sharpening Stone,400/1000.2000/5000 Doub...",https://m.media-amazon.com/images/I/61uWVjhezJ...,https://www.amazon.ca/dp/B07KMT96S2,4.4,366,29.99,35.99,Industrial Scientific,False,50


In [3]:
# init lemmatizer to avoid slow performance
mystem = Mystem() 

def word_tokenize_clean(doc: str, stop_words: list):
    '''
    tokenize from string to list of words
    '''

    # split into lower case word tokens \w lemmatization
    tokens = list(set(mystem.lemmatize(doc.lower())))
  
    # remove tokens that are not alphabetic (including punctuation) and not a stop word
    tokens = [word for word in tokens if word.isalpha() and not word in stop_words \
              not in list(punctuation)]
    return tokens

In [4]:
df = df.reset_index().rename(columns = {'index': 'model_index'})
df['model_index'] = df['model_index'].astype(str)
df.head(5)

,model_index,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,categoryName,isBestSeller,boughtInLastMonth
0,0,B07CV4L6HX,"Green Leaf WW3D Wonder Extension Cord Winder, ...",https://m.media-amazon.com/images/I/81cRe0AVC4...,https://www.amazon.ca/dp/B07CV4L6HX,4.4,2876,47.69,0.00,Industrial Scientific,False,0
1,1,B09N1HGY74,8pcs Toilet Seat Bumpers Universal Toilet Repl...,https://m.media-amazon.com/images/I/512CpB-EP5...,https://www.amazon.ca/dp/B09N1HGY74,3.8,55,10.99,0.00,Industrial Scientific,False,100
2,2,B087P7538J,YaeCCC 19 Pcs Hole Saw Kit 3/4''(19mm)- 6''(15...,https://m.media-amazon.com/images/I/71P+qLFDFc...,https://www.amazon.ca/dp/B087P7538J,4.0,126,25.99,27.99,Industrial Scientific,False,50
3,3,B0822FF7YQ,LLPT Butyl Putty Tape White 1 Inch x 33Ft Tigh...,https://m.media-amazon.com/images/I/61wfWLZRn-...,https://www.amazon.ca/dp/B0822FF7YQ,4.5,1936,21.99,30.99,Industrial Scientific,False,100
4,4,B08NTR8NN4,"Lightbeam 16"" Long Stem Deep Fry Thermometer w...",https://m.media-amazon.com/images/I/61qJG-y1He...,https://www.amazon.ca/dp/B08NTR8NN4,4.2,46,18.99,0.00,Industrial Scientific,False,100


In [5]:
products_mapper = dict(zip(df['title'].str.lower(), df['model_index'].astype(int)))
products_mapper

{'green leaf ww3d wonder extension cord winder, gray, 150 foot': 0,
 '8pcs toilet seat bumpers universal toilet replacement bumper for bidet attachment toilet seat bumper pads with strong adhesive for home, hotel, hospital': 1,
 "yaeccc 19 pcs hole saw kit 3/4''(19mm)- 6''(152mm) full set in case with mandrels, hex key and install plate for soft wood, pvc board and plastic plate drilling": 2,
 'llpt butyl putty tape white 1 inch x 33ft tight seal for leak proof rv repair window glass boat sealing roof pipe patching(wst233)': 3,
 'lightbeam 16" long stem deep fry thermometer with clip, instant read 2" dial oil thermometer meat thermometer for deep frying, broiling, bbq & turkey ...': 4,
 'etguuds white usb c to usb c cable [10ft, 2-pack], 60w fast charging type c to type c charger cable for samsung galaxy s23 s22 s21 s20 ultra 5g, z flip/fold 4 3, note 20, pixel 7 6 pro & usb-c laptop': 5,
 'reaqer cpap hose holder hanger for preventing tube leakage and tangle adjustable and sturdy': 6,

In [6]:
# предварительная обработка путем удаления несимвольных данных, стоп-слов
tags_corpus = df['title'].values
tags_corpus = [re.sub('[!/()0-9]', '', str(x)) for x in tags_corpus]
stop_words = stopwords.words('english')

tags_corpus[:1]

['Green Leaf WWD Wonder Extension Cord Winder, Gray,  Foot']

In [7]:
tags_doc = [TaggedDocument(words = word_tokenize_clean(D, stop_words), tags = [str(i)]) for i, D in enumerate(tags_corpus)]
tags_doc[1]

TaggedDocument(words=['bidet', 'seat', 'universal', 'hotel', 'strong', 'home', 'replacement', 'attachment', 'pcs', 'bumpers', 'adhesive', 'toilet', 'bumper', 'pads', 'hospital'], tags=['1'])

In [8]:
VEC_SIZE = 50 # длина вектора для каждого фильма
ALPHA = .02 # параметр обучения модели
MIN_ALPHA = .00025 # параметр обучения модели
MIN_COUNT = 4 # минимальное количество вхождений слова в словарь
EPOCHS = 20 # количество тренировок

In [9]:
# инициализировать модель
model = Doc2Vec(vector_size = VEC_SIZE,
                alpha = ALPHA, 
                min_alpha = MIN_ALPHA,
                min_count = MIN_COUNT,
                dm = 0)

In [ ]:
# создайте словарь из всех документов по тегам
model.build_vocab(tags_doc)

# обучить модель
model.train(tags_doc,
            total_examples = model.corpus_count,
            epochs = EPOCHS)

In [ ]:
"""product_id = products_mapper['oxiclean versatile stain remover, 275 loads']
product_id"""

In [13]:
"""products_vectors = model.dv.vectors
products_embeddings = products_vectors[product_id]"""

'products_vectors = model.dv.vectors\nproducts_embeddings = products_vectors[product_id]'

In [15]:
"""similars = model.docvecs.most_similar(positive = [products_embeddings], topn = 20)
output = pd.DataFrame(similars, columns = ['model_index', 'model_score'])
output.head()"""

"similars = model.docvecs.most_similar(positive = [products_embeddings], topn = 20)\noutput = pd.DataFrame(similars, columns = ['model_index', 'model_score'])\noutput.head()"

In [19]:
import numpy as np


product_title_list = df['title'].fillna("").tolist()
title_to_index = {title: idx for idx, title in enumerate(product_title_list)}

def recommend(product_index: int = -1, top_n: int = 20, fromvalue: float = None, tovalue: float = None, category_list: list = None):
    if product_index != -1:   
        # Step 1: Get embeddings
        products_embeddings = model.dv[str(product_index)]

        # Step 2: Get broader pool of similar products
        similars = model.dv.most_similar(positive=[products_embeddings], topn=500)
        similar_df = pd.DataFrame(similars, columns=['model_index', 'model_score'])
        similar_df['model_index'] = similar_df['model_index'].astype(int)

        # Step 3: Merge with product metadata
        full_df = df.copy()
        full_df['model_index'] = full_df.index
        merged = pd.merge(similar_df, full_df, on='model_index', how='left')

        # Step 4: Filter by same category
        target_category = df.loc[product_index, 'categoryName']

        # Step 5: Convert numeric fields
        merged['stars'] = pd.to_numeric(merged['stars'], errors='coerce').fillna(0)
        merged['reviews'] = pd.to_numeric(merged['reviews'], errors='coerce').fillna(0)
        merged['boughtInLastMonth'] = pd.to_numeric(merged['boughtInLastMonth'], errors='coerce').fillna(0)

        # Step 6: Optional price filter
        if fromvalue is not None and tovalue is not None:
            merged = merged[(merged['price'] >= fromvalue) & (merged['price'] <= tovalue)]

        # Step 7: Add discount and normalize features
        
        merged['reviews_log'] = np.log1p(merged['reviews'])

        # Step 8: Rank scoring
        merged['rank_score'] = (
            merged['model_score'] * 0.5 +
            merged['stars'] * 0.2 +
            merged['reviews_log'] * 0.1 +
            merged['boughtInLastMonth'] * 0.1
        )

        # Step 9: Sort and return
        result = merged.sort_values(by='rank_score', ascending=False).head(top_n)

        return result[[
            'title', 'stars', 'reviews',
            'boughtInLastMonth', 'rank_score', 'productURL', 'categoryName'
        ]]
    else:
        full_df = df.copy()

        # Filter by category list
        filtered = full_df[full_df['categoryName'].isin(category_list)]

        # Clean and convert
        filtered['price'] = pd.to_numeric(filtered['price'], errors='coerce')
        filtered['stars'] = pd.to_numeric(filtered['stars'], errors='coerce').fillna(0)
        filtered['reviews'] = pd.to_numeric(filtered['reviews'], errors='coerce').fillna(0)
        filtered['boughtInLastMonth'] = pd.to_numeric(filtered['boughtInLastMonth'], errors='coerce').fillna(0)

        # Optional price filtering
        if fromvalue is not None and tovalue is not None:
            filtered = filtered[(filtered['price'] >= fromvalue) & (filtered['price'] <= tovalue)]

        # Add review log
        filtered['reviews_log'] = np.log1p(filtered['reviews'])

        # Score products (no embedding here)
        filtered['rank_score'] = (
            filtered['stars'] * 0.4 +
            filtered['reviews_log'] * 0.3 +
            filtered['boughtInLastMonth'] * 0.3
        )

        result = filtered.sort_values(by='rank_score', ascending=False).head(top_n)

        return result[[
            'title', 'stars', 'reviews', 'boughtInLastMonth', 'rank_score', 'productURL', 'categoryName'
        ]]

# No strict price filter
#recommend(product_index=-1, category_list=["Industrial  Scientific", "Baby  Child Care Products"])





product_index = title_to_index["Heat Insulation Silicone Repair Mat with Scale Ruler and Screw Position for Soldering Iron, Phone and Computer Repair Size:17.9 x 11.8 Inches"]

recommend(product_index=product_index)


,title,stars,reviews,boughtInLastMonth,rank_score,productURL,categoryName
129,Crest 3D White Whitestrips Professional Effect...,4.5,1593,10000,1002.132925,https://www.amazon.ca/dp/B07QHPWHQR,Beauty
113,Rosemary Mint Scalp & Hair Strengthening Oil M...,4.4,834,10000,1002.048337,https://www.amazon.ca/dp/B09BMD8SFV,Beauty
445,Kitsch Satin Heatless Curling Set - Hair Rolle...,4.2,7348,9000,902.225194,https://www.amazon.ca/dp/B09S4PVZ81,Beauty
455,Revlon RVDR5222F One-Step Volumizer and Ionic ...,4.5,49838,5000,502.476604,https://www.amazon.ca/dp/B01HZ5K8UE,Beauty
30,CeraVe Eye Repair Cream | Under Eye Cream for ...,4.2,53985,4000,402.425751,https://www.amazon.ca/dp/B00JJPMXDO,Beauty
313,"Lansinoh HPA Lanolin Nipple Cream, Safe for Ba...",4.7,5529,4000,402.296936,https://www.amazon.ca/dp/B00BLA3EPQ,Beauty
211,Pampers Aqua Pure Sensitive Baby Wipes 6X Pop-...,4.7,4454,4000,402.275494,https://www.amazon.ca/dp/B077D3JKVR,Beauty
81,"Retinol Eye Stick, Retinol Eye Cream for Dark ...",4.3,6126,4000,402.227797,https://www.amazon.ca/dp/B0B4PW6ZKY,Beauty
437,"L'Oréal Paris Telescopic Mascara, Eyelash Leng...",4.4,53456,3000,302.463634,https://www.amazon.ca/dp/B004BD107O,Beauty
496,"e.l.f. Instant Lift Brow Pencil, Dual-Ended Pr...",4.3,34431,3000,302.399578,https://www.amazon.ca/dp/B00WAJAKOU,Beauty


In [ ]:
top_20_categories = df["categoryName"].nunique()
print(top_20_categories)


266


In [ ]:
df["stars"]

0          4.4
1          3.8
2          4.0
3          4.5
4          4.2
          ... 
2165921    0.0
2165922    0.0
2165923    0.0
2165924    0.0
2165925    0.0
Name: stars, Length: 2165926, dtype: float64

In [ ]:
df["boughtInLastMonth"].sort_values(ascending=False)

1014818    20000
1565309    20000
685063     20000
267603     10000
1014819    10000
           ...  
753619         0
753618         0
753617         0
753616         0
2165925        0
Name: boughtInLastMonth, Length: 2165926, dtype: int64